In [3]:
%%time
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import os
import datetime as dt
import time
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.2f}'.format

#Указываем название файла с распределением
filename = 'data\Report_389_21.01.2020_277_93_23.01.2020_09-31-01.xlsx'

#Указываем дату начала расчета
#calc_date = dt.datetime.today().strftime('%Y-%m-%d')
calc_date = dt.date(2020, 1, 1).strftime('%Y-%m-%d')

# Загружаем данные из распределения СОП
df_supplychain_raw = pd.read_excel(filename, sheet_name= 'Supply Chain')
df_inventory_raw = pd.read_excel(filename, sheet_name= 'Inventory')
df_demand_raw = pd.read_excel(filename, sheet_name= 'Demand')

#возможности
df_nb_option = pd.read_excel('data\Возможность НБ.xlsx', sheet_name= 'Лист1')
df_npz_option = pd.read_excel('data\Возможность НПЗ.xlsx', sheet_name= 'Лист1')

#Загружаем справочники
df_postavshik_sprav = pd.read_csv('data\sprav postavshikov.csv', sep = '|', encoding = 'utf-8')[['НПЗ', 'Поставщик', 'Время от покупки до отгрузки']]
df_postavshik_sprav = df_postavshik_sprav.drop_duplicates()
df_np_sprav =  pd.read_csv('data//np sprav.csv', sep = '|', encoding = 'utf-8')[['Тип НП', 'НП']]
df_np_sprav.columns = ['Тип НП', 'Продукт']
df_np_sprav = df_np_sprav.drop_duplicates()
df_vremya_v_puti = pd.read_csv('data\jd tariffs.csv', sep = '|', encoding = 'utf-8')[['НБ', 'Базис покупки', 'Расчетное время (дни)']]
df_vremya_v_puti = df_vremya_v_puti.drop_duplicates()
df_vremya_v_puti.columns = ['НБ', 'Базис', 'Расчетное время (дни)']
df_kratnost = pd.read_excel('data\kratnost.xlsx', sheet_name= 'kratnost')

#Рассчитываем продолжительность периода
dates  = pd.DatetimeIndex(df_demand_raw['Период реализации']).drop_duplicates()
period_len = []

#Определяем декадное или месячное распределение поступило на формирование графика
df_percalc = pd.DataFrame()
df_percalc['dates'] = dates
df_percalc['dates_shift'] = df_percalc['dates'].shift()
df_percalc['dates_len'] = df_percalc['dates'] - df_percalc['dates_shift']
decade_or_month = df_percalc['dates_len'].min().days

#Для каждого периода времени определяем длительность
if decade_or_month <= 27:

    for i in dates:
        if i.day == 21:
            period_len.append(((i + MonthEnd()) - i).days +1)
        else:
            period_len.append(10)
            
else:
    for i in dates:
        period_len.append((((i + MonthEnd()) - i)).days + 1)

df_period_lenght = pd.DataFrame(period_len, dates, columns= ['period_len'])

#Рассчитываем ежедневную потребность в ресурсе на каждой из НБ

df_demand = df_demand_raw.groupby(['Период реализации', 'НБ', 'Продукт'])[['Удовлетворенный спрос']].sum()
df_demand = df_demand.join(df_period_lenght)
df_demand.reset_index(inplace = True)


df_inventory_tmp = df_inventory_raw[['Период', 'НБ', 'Продукт', 'Остатки на конец физ.', 'Остатки на начало']]
df_inventory_tmp.columns = ['Период реализации', 'НБ', 'Продукт', 'Остатки на конец физ.', 'wh_1']

df_demand = pd.merge(df_demand, df_inventory_tmp, how ='outer', on= ['Период реализации', 'НБ', 'Продукт'])


df_demand.fillna(0, inplace = True)

df_demand['per_lag'] = 0
df_demand = pd.merge(df_demand, df_np_sprav , how ='left', on= 'Продукт')

df_demand = pd.merge(df_demand, df_kratnost[df_kratnost['Тип транспортировки'] == 'ЖД'][['Кратность отгрузки, т', 'Тип НП']]
                   ,how ='left', on= 'Тип НП')
df_demand = pd.merge(df_demand, df_nb_option, how ='left', on= ['Продукт', 'НБ'])
df_demand[['Возможность НБ']] =  df_demand[['Возможность НБ']].fillna(9999)


df_demand['ССР'] = df_demand['Удовлетворенный спрос']/df_demand['period_len']
df_demand.fillna(0, inplace = True)

df_demand['daily_goal'] = df_demand['ССР'] + df_demand[ 'Остатки на конец физ.']

df_demand['zero_col'] = 0
df_demand['daily_df_demand_raw'] = df_demand['daily_goal'] - df_demand['wh_1']

#Дневная потребность корректируется на Возможность НБ и на 0 для отрицательных
df_demand['daily_demand'] = df_demand[['daily_df_demand_raw', 'zero_col']].max(axis = 1)
df_demand['daily_demand'] = df_demand[['daily_demand', 'Возможность НБ']].min(axis = 1)
df_demand['daily_supply'] = 0
df_demand['wh_2'] = df_demand['wh_1']

#Объем поставки по каждой цепочке
df_supply= df_supplychain_raw.groupby(['Период прихода','Поставщик', 'НПЗ', 'Базис' ,'Регион' , 'НБ', 'Продукт'])[['Объем']].sum()
df_supply= df_supply.join(df_supplychain_raw.groupby(['Период прихода','Поставщик', 'НПЗ', 'Базис' ,'Регион' , 'НБ', 'Продукт'])[[ 'Удельная маржа']].mean())

df_period_lenght.reset_index(inplace = True)

df_supply.reset_index(inplace = True)
df_supply= pd.merge(df_supply, df_period_lenght, left_on= 'Период прихода', right_on='Период реализации')
df_supply.drop(['Период реализации'], axis = 1, inplace = True)

df_supply= pd.merge(df_supply, df_np_sprav, how ='left', on= 'Продукт')
df_supply= pd.merge(df_supply, df_kratnost[df_kratnost['Тип транспортировки'] == 'ЖД'][['Кратность отгрузки, т', 'Тип НП']]
                   ,how ='left', on= 'Тип НП')

df_supply= pd.merge(df_supply, df_postavshik_sprav,  how= 'left', on= ['Поставщик', 'НПЗ'])
df_supply= pd.merge(df_supply, df_vremya_v_puti, how= 'left',  on= ['Базис', 'НБ'])

df_supply.fillna(0, inplace = True)
df_supply['Расчетное время (дни)']  = np.where(df_supply['НПЗ'] == 'КС',11,df_supply['Расчетное время (дни)'])

df_supply['Расчетное время (дни)'] = pd.to_timedelta(df_supply['Расчетное время (дни)'], 'd')
df_supply['Время от покупки до отгрузки'] = pd.to_timedelta(df_supply['Время от покупки до отгрузки'], 'd')

df_supply['Дата прихода'] = df_supply['Период прихода']
df_supply['Дата отгрузки'] = df_supply['Дата прихода'] - df_supply['Расчетное время (дни)']
df_supply['Дата закупки'] = df_supply['Дата отгрузки'] - df_supply['Время от покупки до отгрузки']
df_supply['per_lag'] = 0
df_supply['daily_ship'] = 0
df_supply['obem_ost'] =  df_supply['Объем']

df_supply.sort_values(['Продукт', 'Период прихода', 'НБ', 'Удельная маржа'], ascending = False, inplace = True)

#рассчитываем рейтинг поставщиков на каждую НБ и НП по периоду
rating = df_supply.groupby(['Продукт' , 'Период прихода', 'НБ'])[ 'Удельная маржа'].rank(ascending = False, method = 'first')
rating.name = 'rating'
df_supply = pd.concat([df_supply, rating], axis = 1)

#Определяем максимальное число поставщиков для одной НБ и НП по период
#Добавляем в таблицу число столбцов равное полученному максимуму
max_rating = int(df_supply.rating.max())
for i in np.array(range(max_rating)) +1:
    df_supply['ship_' + str(i)] = 0

p_start = time.time()
time_time_2 = p_start

df_demand_for_loop = df_demand.copy()
df_supply_for_loop = df_supply.copy()

#Основной расчет
#Цикл для каждого отдельного дня в периоде 
for i in np.array(range(max(period_len))) + 1:
#for i in range(1,3):
    loop_sup = df_supply_for_loop[(df_supply_for_loop['per_lag'] == (i -1))].copy()
    loop_sup['per_lag'] = i
    loop_sup['daily_ship'] = 0
    loop_sup['timedelta_per_lag'] = pd.to_timedelta(loop_sup['per_lag'] - 1, 'd')
    loop_sup['Дата прихода'] = loop_sup['Период прихода'] + loop_sup['timedelta_per_lag']
    loop_sup['Дата отгрузки'] = loop_sup['Дата прихода'] - loop_sup['Расчетное время (дни)']
    loop_sup['Дата закупки'] = loop_sup['Дата отгрузки'] - loop_sup['Время от покупки до отгрузки']
    
    loop_dem = df_demand_for_loop[(df_demand_for_loop['per_lag'] == (i -1))].copy()
    loop_dem['per_lag'] = i
    loop_dem['wh_1'] = loop_dem['wh_2']
    loop_dem['daily_df_demand_raw'] = loop_dem['daily_goal'] - loop_dem['wh_1']
    loop_dem['daily_demand'] = loop_dem[['daily_df_demand_raw', 'zero_col']].max(axis = 1)
    loop_dem['daily_demand'] = loop_dem[['daily_demand', 'Возможность НБ']].min(axis = 1)
    loop_dem['daily_supply'] = 0
    
    #Цикл для каждого поставщика в пределахх максимального рейтинга 
    for m in np.array(range(max_rating)) +1:
    #for m in range(1,2):
        loop_sup = pd.merge(loop_sup, loop_dem[['Период реализации', 'НБ', 'Продукт', 'daily_demand']]
                  , left_on= ['Период прихода', 'НБ', 'Продукт'], right_on=['Период реализации', 'НБ', 'Продукт'])
        #if m  == 1:cl
        loop_sup['ship_' + str(m)] =  np.where(loop_sup['Дата закупки'] < calc_date, 0 ,
                                               np.where(loop_sup['period_len'] < i, 0, 
                                                        np.where((loop_sup['rating'] == m), 
                                                                 np.where((loop_sup['daily_demand'] >= loop_sup['obem_ost']),
                                                                          round(loop_sup['obem_ost']/loop_sup['Кратность отгрузки, т']) * loop_sup['Кратность отгрузки, т'],round(loop_sup['daily_demand']/ loop_sup['Кратность отгрузки, т']) * loop_sup['Кратность отгрузки, т']), 0)))
        
        a  = pd.DataFrame(loop_sup.groupby(['Период прихода', 'НБ', 'Продукт'])['ship_' + str(m)].sum())
        a.reset_index(inplace = True)
        
        loop_dem = pd.merge(loop_dem, a,
                            left_on=['Период реализации', 'НБ', 'Продукт'],right_on= ['Период прихода', 'НБ', 'Продукт'])
        del a
        
        loop_sup['obem_ost'] = loop_sup['obem_ost'] - loop_sup['ship_' + str(m)]
        loop_sup['daily_ship'] = loop_sup['daily_ship'] + loop_sup['ship_' + str(m)]
        loop_dem['daily_demand'] = loop_dem['daily_demand'] - loop_dem['ship_' + str(m)]
        loop_dem['daily_supply'] = loop_dem['daily_supply'] + loop_dem['ship_' + str(m)]
                                                         
    #Дропаем рабочие поля
        loop_dem.drop(['Период прихода'] + list(loop_dem.filter(regex='ship_')), axis = 1, inplace = True)    
        loop_sup.drop(['daily_demand']+ list(loop_sup.filter(regex='Период реализации')), axis = 1, inplace = True)  
    #считаем остаток на конец дня
    loop_dem['wh_2'] = loop_dem['wh_1'] - loop_dem['ССР'] + loop_dem['daily_supply']
    
    df_supply_for_loop = pd.concat([df_supply_for_loop, loop_sup])
    df_demand_for_loop = pd.concat([df_demand_for_loop, loop_dem])
    
    del loop_sup, loop_dem
    
    time_time = time.time()
    print('День %s  минут этапа %f минут итого %f'  
              % (i,  (time_time - time_time_2)/60, (time_time - p_start)/60))
    time_time_2 = time_time
    
df_supply_for_loop.reset_index(inplace = True)
df_supply_for_loop.drop(list(df_supply_for_loop.filter(regex='ship_')) + ['index'], axis = 1, inplace = True)

#Проверка на соответствие результата входным данным
df_check_volume = pd.DataFrame(df_supplychain_raw.groupby(['Период прихода', 'Базис', 'НБ', 'Продукт'])['Объем'].sum())
df_check_volume = df_check_volume.join(df_supply_for_loop.groupby(['Период прихода', 'Базис', 'НБ', 'Продукт'])['daily_ship'].sum())
df_check_volume = df_check_volume.join(df_supply_for_loop.groupby(['Период прихода', 'Базис', 'НБ', 'Продукт'])['Кратность отгрузки, т'].mean())
df_check_volume.columns = ['v_raw', 'v_calc', 'crat']
df_check_volume['diff'] = (df_check_volume['v_raw'] - df_check_volume['v_calc']).abs()
#Проверка на отклонение объемов в графике и в репорте
print('\nПроверка на отклонение объемов в графике и в репорте\n\n' ,df_check_volume[df_check_volume['diff']>=df_check_volume['crat']])

#Результаты записываем в файлы
df_demand_for_loop.to_excel('output\demand.xlsx', index=None)
df_supply_for_loop.to_excel('output\supply_schedule.xlsx', index = None)

День 1  минут этапа 0.005206 минут итого 0.005206
День 2  минут этапа 0.007688 минут итого 0.012894
День 3  минут этапа 0.153245 минут итого 0.166139


MemoryError: 